In [4]:
import os

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [8]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [9]:
df.head()

,Product,Review
0,Queen Size Sheet Set\t,I ordered the bed and it looks nice
1,Waterproof Phone Pouch,I love it althought opening it is a bit diffi...
2,Luxury Air Mattress,The mattress had a small hole in the top of i...


In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
llm = ChatOpenAI(temperature=0.9)

In [18]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [19]:
chain = LLMChain(llm=llm, prompt=prompt)

In [21]:
product = 'Queen Size Bed'
chain.run(product)

'One possible name to describe a company that makes Queen Size Beds could be "Regal Rests"'

In [23]:
from langchain.chains import SimpleSequentialChain

In [25]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [26]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company_name}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [27]:
simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [28]:
simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
RoyalRest
RoyalRest is a luxury bedding company that offers high-quality mattresses and sleep accessories designed for ultimate comfort and rejuvenation.

> Finished chain.


'RoyalRest is a luxury bedding company that offers high-quality mattresses and sleep accessories designed for ultimate comfort and rejuvenation.'

In [55]:
from langchain.chains import SequentialChain

In [56]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:"
    "\n\n {Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [57]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarise the following review in 1 sentence:"
    "\n\n {English_Review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [58]:
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:"
    "\n\n {Review}"
)

chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [59]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following"
    "summary in the specified language"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

chain_four = LLMChain(llm=llm, prompt=fourth_prompt)

In [63]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "language"],
    verbose=True
)

In [64]:
review = df.Review[1]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': ' I love it althought opening it is a bit difficult',
 'English_Review': 'I love it although opening it is a bit difficult.',
 'summary': 'The reviewer expressed their love for the product, but mentioned that the opening process may be a bit challenging.',
 'language': 'The language of the review appears to be English.'}